In [1]:
import astropy
from astropy.table import Table,Column
import numpy as np
import matplotlib.pyplot as plt
import glob
import subprocess
import os

In [2]:
def pop_list(ls,indices):
    for n,ind in enumerate(indices):
        ls.pop(ind)
    return(ls)

In [3]:
photz = Table.read('data/data_release/final_cross_match_catalogue-v0.5.fits')

In [4]:
photz[0]

Source_Name,RA,DEC,E_RA,E_DEC,Total_flux,E_Total_flux,Peak_flux,E_Peak_flux,S_Code,Maj,Min,PA,E_Maj,E_Min,E_PA,DC_Maj,DC_Min,DC_PA,Isl_rms,FLAG_WORKFLOW,Prefilter,NoID,lr_fin,optRA,optDec,LGZ_Size,LGZ_Width,LGZ_PA,Assoc,Assoc_Qual,Art_prob,Blend_prob,Hostbroken_prob,Imagemissing_prob,Zoom_prob,Created,Position_from,Renamed_from,FLAG_OVERLAP_RADIO,flag_clean_radio,ALPHA_J2000,DELTA_J2000,FLAG_DEEP,flag_clean,ID_OPTICAL,ID_SPITZER,ID,FLAG_OVERLAP,EBV,FUV_flux_corr,FUV_fluxerr_corr,FUV_mag_corr,FUV_magerr_corr,NUV_flux_corr,NUV_fluxerr_corr,NUV_mag_corr,NUV_magerr_corr,u_flux_corr,u_fluxerr_corr,u_mag_corr,u_magerr_corr,Bw_flux_corr,Bw_fluxerr_corr,Bw_mag_corr,Bw_magerr_corr,R_flux_corr,R_fluxerr_corr,R_mag_corr,R_magerr_corr,I_flux_corr,I_fluxerr_corr,I_mag_corr,I_magerr_corr,z_flux_corr,z_fluxerr_corr,z_mag_corr,z_magerr_corr,z_Subaru_flux_corr,z_Subaru_fluxerr_corr,z_Subaru_mag_corr,z_Subaru_magerr_corr,y_flux_corr,y_fluxerr_corr,y_mag_corr,y_magerr_corr,J_flux_corr,J_fluxerr_corr,J_mag_corr,J_magerr_corr,H_flux_corr,H_fluxerr_corr,H_mag_corr,H_magerr_corr,K_flux_corr,K_fluxerr_corr,K_mag_corr,K_magerr_corr,Ks_flux_corr,Ks_fluxerr_corr,Ks_mag_corr,Ks_magerr_corr,ch1_flux_corr,ch1_fluxerr_corr,ch1_mag_corr,ch1_magerr_corr,ch2_flux_corr,ch2_fluxerr_corr,ch2_mag_corr,ch2_magerr_corr,ch3_flux_corr,ch3_fluxerr_corr,ch3_mag_corr,ch3_magerr_corr,ch4_flux_corr,ch4_fluxerr_corr,ch4_mag_corr,ch4_magerr_corr,Separation,help_id,RA_HELP,DEC_HELP,CLASS_STAR,F_MIPS_24,FErr_MIPS_24_u,FErr_MIPS_24_l,Bkg_MIPS_24,Sig_conf_MIPS_24,Rhat_MIPS_24,n_eff_MIPS_24,Pval_res_24,flag_mips_24,F_PACS_100,FErr_PACS_100_u,FErr_PACS_100_l,F_PACS_160,FErr_PACS_160_u,FErr_PACS_160_l,Bkg_PACS_100,Bkg_PACS_160,Sig_conf_PACS_100,Sig_conf_PACS_160,Rhat_PACS_100,Rhat_PACS_160,n_eff_PACS_100,n_eff_PACS_160,Pval_res_100,Pval_res_160,flag_PACS_100,flag_PACS_160,F_SPIRE_250,FErr_SPIRE_250_u,FErr_SPIRE_250_l,F_SPIRE_350,FErr_SPIRE_350_u,FErr_SPIRE_350_l,F_SPIRE_500,FErr_SPIRE_500_u,FErr_SPIRE_500_l,Bkg_SPIRE_250,Bkg_SPIRE_350,Bkg_SPIRE_500,Sig_conf_SPIRE_250,Sig_conf_SPIRE_350,Sig_conf_SPIRE_500,Rhat_SPIRE_250,Rhat_SPIRE_350,Rhat_SPIRE_500,n_eff_SPIRE_250,n_eff_SPIRE_500,n_eff_SPIRE_350,Pval_res_250,Pval_res_350,Pval_res_500,flag_spire_250,flag_spire_350,flag_spire_500,Z_BEST,Z_BEST_SOURCE,Z_SPEC,Z_SOURCE,Z_QUAL,RA_ZSPEC,DEC_ZSPEC,REL,OBJID,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,nfilt_eazy,nfilt_atlas,nfilt_ananna,chi_r_best,chi_r_stellar,stellar_type,AGN,optAGN,IRAGN,AGN_ZSPEC,XrayAGN,XrayFlux_0.5-2,XrayHardness,hp_idx_11,hp_depth_cluster,tot_corr_z_Subaru,tot_corr_err_z_Subaru,tot_corr_z,tot_corr_err_z,zmodel,chi_best,Mass_median,Mass_l68,Mass_u68,SFR_median,SFR_l68,SFR_u68,Nfilts,u_rest,Bw_rest,R_rest,I_rest,z_rest,z_Subaru_rest,y_rest,J_rest,H_rest,K_rest,Ks_rest,ch1_rest,ch2_rest,ch3_rest,ch4_rest
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bytes22,float64,float64,float64,float64,float64,float64,float64,float64,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,bytes22,bytes17,bytes22,int64,int64,float64,float64,float64,int64,int64,int64,float64,int64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float

In [5]:
cigale_input = photz.copy()

In [6]:
for name in cigale_input.colnames:
    if '_corr' in name:
        cigale_input.rename_column(name,name.replace('_corr',''))

In [9]:
filters_file = open('data/photz/filters/filter.bootes_mbrown_2014a.res.info','r')
#filters_file = open('/beegfs/lofar/deepfields/science_ready_catalogs/Lockman-SWIRE_filters.res.info','r')
filters = []
for line in filters_file:
    temp = line.split(' ')
    filters.append(temp)
filters_file.close()

for n,line in enumerate(filters):
    if n!=32:
        filters[n] = line[0:-1]
    else:
        filters[n][-1] = filters[n][-1].replace('\n','')
    filters[n] = pop_list(filters[n],[1 for m in range(len(filters[n])-2)])
    filters[n][-1] = filters[n][-1].replace('.filter','')
#ignore the first two filters as they are SWIRE channel 1 and 2 and i want to use SERVS channel 1 and 2 as
#they have deeper observations
filters = filters[2:]

In [10]:
filters

[['3', 'IRAC_ch1_total_response'],
 ['4', 'IRAC_ch2_total_response'],
 ['5', 'IRAC_ch3_total_response'],
 ['6', 'IRAC_ch4_total_response'],
 ['7', 'I_atmccd'],
 ['8', 'K_atmccd'],
 ['9', 'LBCBlue_Uspec'],
 ['10', 'LBCR_Y'],
 ['11', 'R_atmccd'],
 ['12', 'U_bessell'],
 ['13', 'V_bessell'],
 ['14', 'Y'],
 ['15', 'fuv'],
 ['16', 'galexFUV'],
 ['17', 'galexNUV'],
 ['18', 'hband'],
 ['19', 'jband'],
 ['20', 'kband'],
 ['21', 'lbtUspec'],
 ['22', 'lbtY'],
 ['23', 'mips24'],
 ['24', 'newfirmH'],
 ['25', 'newfirmJ'],
 ['26', 'newfirmKs'],
 ['27', 'nuv'],
 ['28', 'sdssg'],
 ['29', 'sdssi'],
 ['30', 'sdssr'],
 ['31', 'sdssu'],
 ['32', 'sdssz'],
 ['33', 'suprime_FDCCD_z'],
 ['34', 'sdssz2']]

In [11]:
#translate_file = open('/beegfs/lofar/deepfields/science_ready_catalogs/LH.filter.translate','r')
translate_file = open('data/photz/filters/brown.zphot.2014.translate','r')
translate = []
for line in translate_file:
    if '#' in line:
        continue
    temp = line.split(' ')
    translate.append(temp)
translate_file.close()

translate = translate[:len(translate)-1]

for n,line in enumerate(translate):
    translate[n][1] = line[1][1:-1]

In [12]:
translate

[['u_flux', '9'],
 ['u_fluxerr', '9'],
 ['Bw_flux', '2'],
 ['Bw_fluxerr', '2'],
 ['R_flux', '11'],
 ['R_fluxerr', '11'],
 ['I_flux', '7'],
 ['I_fluxerr', '7'],
 ['z_flux', '34'],
 ['z_fluxerr', '34'],
 ['z_Subaru_flux', '33'],
 ['z_Subaru_fluxerr', '33'],
 ['y_flux', '10'],
 ['y_fluxerr', '10'],
 ['J_flux', '19'],
 ['J_fluxerr', '19'],
 ['H_flux', '18'],
 ['H_fluxerr', '18'],
 ['K_flux', '8'],
 ['K_fluxerr', '8'],
 ['Ks_flux', '20'],
 ['Ks_fluxerr', '20'],
 ['ch1_flux', '3'],
 ['ch1_fluxerr', '3'],
 ['ch2_flux', '4'],
 ['ch2_fluxerr', '4'],
 ['ch3_flux', '5'],
 ['ch3_fluxerr', '5'],
 ['ch4_flux', '6'],
 ['ch4_fluxerr', '6']]

In [13]:
for n,line in enumerate(translate):
    print(line)
    colname = line[0]
    filt_num = int(line[1])
    filt_name = ''
    for m,filt in enumerate(filters):
        if int(filt[0])==filt_num:
            print(filt)
            filt_name = filt[1]
    if filt_name=='':
        continue
    if 'err' in colname:
        filt_name = filt_name+'_err'
    #print(colname)
    #print(filt_name)
    print('colname is: {}'.format(colname))
    print('filter name is: {}'.format(filt_name))
    cigale_input.rename_column(colname,filt_name)

['u_flux', '9']
['9', 'LBCBlue_Uspec']
colname is: u_flux
filter name is: LBCBlue_Uspec
['u_fluxerr', '9']
['9', 'LBCBlue_Uspec']
colname is: u_fluxerr
filter name is: LBCBlue_Uspec_err
['Bw_flux', '2']
['Bw_fluxerr', '2']
['R_flux', '11']
['11', 'R_atmccd']
colname is: R_flux
filter name is: R_atmccd
['R_fluxerr', '11']
['11', 'R_atmccd']
colname is: R_fluxerr
filter name is: R_atmccd_err
['I_flux', '7']
['7', 'I_atmccd']
colname is: I_flux
filter name is: I_atmccd
['I_fluxerr', '7']
['7', 'I_atmccd']
colname is: I_fluxerr
filter name is: I_atmccd_err
['z_flux', '34']
['34', 'sdssz2']
colname is: z_flux
filter name is: sdssz2
['z_fluxerr', '34']
['34', 'sdssz2']
colname is: z_fluxerr
filter name is: sdssz2_err
['z_Subaru_flux', '33']
['33', 'suprime_FDCCD_z']
colname is: z_Subaru_flux
filter name is: suprime_FDCCD_z
['z_Subaru_fluxerr', '33']
['33', 'suprime_FDCCD_z']
colname is: z_Subaru_fluxerr
filter name is: suprime_FDCCD_z_err
['y_flux', '10']
['10', 'LBCR_Y']
colname is: y_flux


In [14]:
fir_cols = ['F_MIPS_24','F_PACS_100','F_PACS_160','F_SPIRE_250','F_SPIRE_350','F_SPIRE_500']
for col in fir_cols:
    col_err = Column(np.zeros(len(cigale_input)),name=col+'err')
    err_up = cigale_input[col.replace('F','FErr')+'_u'] - cigale_input[col]
    err_low = cigale_input[col] - cigale_input[col.replace('F','FErr')+'_l']
    mask = err_up>err_low
    col_err[mask] = err_up[mask]
    col_err[~mask] = err_low[~mask]
    cigale_input.add_column(col_err)

/its/home/im281/.conda/envs/herschelhelp/lib/python3.6/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)


In [15]:
cigale_input.rename_column('z1_median','redshift')
cigale_input.rename_column('Source_Name','id')
cigale_input.rename_column('F_MIPS_24','mips_24')
cigale_input.rename_column('F_MIPS_24err','mips_24_err')
cigale_input.rename_column('F_PACS_100','pacs_green')
cigale_input.rename_column('F_PACS_100err','pacs_green_err')
cigale_input.rename_column('F_PACS_160','pacs_red')
cigale_input.rename_column('F_PACS_160err','pacs_red_err')
cigale_input.rename_column('F_SPIRE_250','spire_250')
cigale_input.rename_column('F_SPIRE_250err','spire_250_err')
cigale_input.rename_column('F_SPIRE_350','spire_350')
cigale_input.rename_column('F_SPIRE_350err','spire_350_err')
cigale_input.rename_column('F_SPIRE_500','spire_500')
cigale_input.rename_column('F_SPIRE_500err','spire_500_err')
cigale_input['mips_24'] = cigale_input['mips_24']/1000
cigale_input['mips_24_err'] = cigale_input['mips_24_err']/1000

In [16]:
cigale_input.colnames

['id',
 'RA',
 'DEC',
 'E_RA',
 'E_DEC',
 'Total_flux',
 'E_Total_flux',
 'Peak_flux',
 'E_Peak_flux',
 'S_Code',
 'Maj',
 'Min',
 'PA',
 'E_Maj',
 'E_Min',
 'E_PA',
 'DC_Maj',
 'DC_Min',
 'DC_PA',
 'Isl_rms',
 'FLAG_WORKFLOW',
 'Prefilter',
 'NoID',
 'lr_fin',
 'optRA',
 'optDec',
 'LGZ_Size',
 'LGZ_Width',
 'LGZ_PA',
 'Assoc',
 'Assoc_Qual',
 'Art_prob',
 'Blend_prob',
 'Hostbroken_prob',
 'Imagemissing_prob',
 'Zoom_prob',
 'Created',
 'Position_from',
 'Renamed_from',
 'FLAG_OVERLAP_RADIO',
 'flag_clean_radio',
 'ALPHA_J2000',
 'DELTA_J2000',
 'FLAG_DEEP',
 'flag_clean',
 'ID_OPTICAL',
 'ID_SPITZER',
 'ID',
 'FLAG_OVERLAP',
 'EBV',
 'FUV_flux',
 'FUV_fluxerr',
 'FUV_mag',
 'FUV_magerr',
 'NUV_flux',
 'NUV_fluxerr',
 'NUV_mag',
 'NUV_magerr',
 'LBCBlue_Uspec',
 'LBCBlue_Uspec_err',
 'u_mag',
 'u_magerr',
 'Bw_flux',
 'Bw_fluxerr',
 'Bw_mag',
 'Bw_magerr',
 'R_atmccd',
 'R_atmccd_err',
 'R_mag',
 'R_magerr',
 'I_atmccd',
 'I_atmccd_err',
 'I_mag',
 'I_magerr',
 'sdssz2',
 'sdssz2_err

In [17]:
cols_keep = ['id',
 'RA',
 'DEC',
 'FUV_flux',
 'FUV_fluxerr',
 'NUV_flux',
 'NUV_fluxerr',
 'LBCBlue_Uspec',
 'LBCBlue_Uspec_err',
 'Bw_flux',
 'Bw_fluxerr',
 'R_atmccd',
 'R_atmccd_err',
 'I_atmccd',
 'I_atmccd_err',
 'sdssz2',
 'sdssz2_err',
 'suprime_FDCCD_z',
 'suprime_FDCCD_z_err',
 'LBCR_Y',
 'LBCR_Y_err',
 'jband',
 'jband_err',
 'hband',
 'hband_err',
 'K_atmccd',
 'K_atmccd_err',
 'kband',
 'kband_err',
 'IRAC_ch1_total_response',
 'IRAC_ch1_total_response_err',
 'IRAC_ch2_total_response',
 'IRAC_ch2_total_response_err',
 'IRAC_ch3_total_response',
 'IRAC_ch3_total_response_err',
 'IRAC_ch4_total_response',
 'IRAC_ch4_total_response_err',
 'mips_24',
 'pacs_green',
 'pacs_red',
 'spire_250',
 'spire_350',
 'spire_500',
 'mips_24_err',
 'pacs_green_err',
 'pacs_red_err',
 'spire_250_err',
 'spire_350_err',
 'spire_500_err',
 'redshift']
cigale_input = cigale_input[cols_keep]

In [18]:
flux_change = [ 'FUV_flux',
 'FUV_fluxerr',
 'NUV_flux',
 'NUV_fluxerr',
 'LBCBlue_Uspec',
 'LBCBlue_Uspec_err',
 'Bw_flux',
 'Bw_fluxerr',
 'R_atmccd',
 'R_atmccd_err',
 'I_atmccd',
 'I_atmccd_err',
 'sdssz2',
 'sdssz2_err',
 'suprime_FDCCD_z',
 'suprime_FDCCD_z_err',
 'LBCR_Y',
 'LBCR_Y_err',
 'jband',
 'jband_err',
 'hband',
 'hband_err',
 'K_atmccd',
 'K_atmccd_err',
 'kband',
 'kband_err',
 'IRAC_ch1_total_response',
 'IRAC_ch1_total_response_err',
 'IRAC_ch2_total_response',
 'IRAC_ch2_total_response_err',
 'IRAC_ch3_total_response',
 'IRAC_ch3_total_response_err',
 'IRAC_ch4_total_response',
 'IRAC_ch4_total_response_err']

for col in flux_change:
    cigale_input[col] = cigale_input[col]/1000

In [ ]:
ini_template = open('pcigale.ini','r')
bins = np.arange(0,6,0.1)
for n in range(len(bins)-1):
    mask = (sed_sample['redshift']>bins[n]) & (sed_sample['redshift']<bins[n+1])
    if np.sum(mask)>0:
        mask_table = sed_sample[mask]
        folder_name = '{}_z_{}'.format(str(bins[n])[:4],str(bins[n+1])[:4])
        fname = 'lofar_sample_cigale_z_{}_to_{}.fits'.format(str(bins[n])[:4],str(bins[n+1])[:4])
        
        os.mkdir('data/cigale_results/sample/{}'.format(folder_name))

        Table.write(mask_table,'data/cigale/sample/{}/{}'.format(folder_name,fname),format='fits',overwrite=True)
        
        ini_template.seek(0)
        new_ini = open('data/cigale_results/sample/{}/pcigale.ini'.format(folder_name),'w+') 
        #create a blank pcigale.ini.spec file so it doesn't crash
        new_ini_spec = open('data/cigale_results/sample/{}/pcigale.ini.spec'.format(folder_name),'w+')
        for line in ini_template:
            if 'data_file' in line:
                line = 'data_file = {}'.format(fname)
                print(line)
            new_ini.write(line)
        new_ini.close()
ini_template.close()

In [27]:
Table.write(cigale_input,'data/cigale_input_Lockman.fits',format='fits',overwrite=True)

/Users/im281/anaconda3/lib/python3.7/site-packages/astropy/table/column.py:981: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return getattr(self.data, op)(other)


IORegistryError: No writer defined for format 'fits' and class 'NoneType'.

The available formats are:

Format Read Write Auto-identify
------ ---- ----- -------------

In [19]:
mask = (cigale_input['redshift']>5.5)
mini_input = cigale_input[mask]
mini_input['redshift'] = np.round(mini_input['redshift'],2)
print(len(mini_input))
mini_input[0]

86


id,RA,DEC,FUV_flux,FUV_fluxerr,NUV_flux,NUV_fluxerr,LBCBlue_Uspec,LBCBlue_Uspec_err,Bw_flux,Bw_fluxerr,R_atmccd,R_atmccd_err,I_atmccd,I_atmccd_err,sdssz2,sdssz2_err,suprime_FDCCD_z,suprime_FDCCD_z_err,LBCR_Y,LBCR_Y_err,jband,jband_err,hband,hband_err,K_atmccd,K_atmccd_err,kband,kband_err,IRAC_ch1_total_response,IRAC_ch1_total_response_err,IRAC_ch2_total_response,IRAC_ch2_total_response_err,IRAC_ch3_total_response,IRAC_ch3_total_response_err,IRAC_ch4_total_response,IRAC_ch4_total_response_err,mips_24,pacs_green,pacs_red,spire_250,spire_350,spire_500,mips_24_err,pacs_green_err,pacs_red_err,spire_250_err,spire_350_err,spire_500_err,redshift
bytes22,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64
ILTJ142428.18+335110.3,216.11743041513492,33.8528786743377,--,--,--,--,0.0011692507102865123,0.00010521505490409677,--,--,--,--,0.00048675061316257176,0.0004938625727962528,0.02481316562125221,0.001176969160551446,0.029298913976487907,0.0009148015691566658,0.03299220152206499,0.0012580193393456635,0.05346635258494216,0.000999659910683474,0.08718144730803518,0.0025614768642129486,0.12226672287112869,0.018918784647994232,0.12265234218305461,0.0035810533259954995,0.08149959669322426,0.0015688355637618638,0.08577336237894098,0.001722204059942409,0.0626695682283681,0.008738929942829547,0.22543093229825503,0.009467908012785147,--,nan,nan,nan,nan,nan,--,nan,nan,nan,nan,nan,6.63


In [21]:
Table.write(mini_input,'cigale/input/rachel_cigale_URGENT_Bootes.fits',format='fits',overwrite=True)

In [22]:
# printing the columns that need to go to CIGALE so they can be added to the .ini file

test = ''
for name in cols_keep:
    test = test + name +', '
print(test)

id, RA, DEC, FUV_flux, FUV_fluxerr, NUV_flux, NUV_fluxerr, LBCBlue_Uspec, LBCBlue_Uspec_err, Bw_flux, Bw_fluxerr, R_atmccd, R_atmccd_err, I_atmccd, I_atmccd_err, sdssz2, sdssz2_err, suprime_FDCCD_z, suprime_FDCCD_z_err, LBCR_Y, LBCR_Y_err, jband, jband_err, hband, hband_err, K_atmccd, K_atmccd_err, kband, kband_err, IRAC_ch1_total_response, IRAC_ch1_total_response_err, IRAC_ch2_total_response, IRAC_ch2_total_response_err, IRAC_ch3_total_response, IRAC_ch3_total_response_err, IRAC_ch4_total_response, IRAC_ch4_total_response_err, mips_24, pacs_green, pacs_red, spire_250, spire_350, spire_500, mips_24_err, pacs_green_err, pacs_red_err, spire_250_err, spire_350_err, spire_500_err, redshift, 
